In [198]:
# importamos las librerias necesarias
import pandas as pd
import faker as fk
import random as rd
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import openpyxl
import sympy as sp

In [199]:
planetas = {
    "Tierra": 9.8,
    "Marte": 3.7,
    "Júpiter": 24.8,
    "Saturno": 9.0,
    "Urano": 8.7,
    "Neptuno": 11.0
}

In [200]:
# Para el apartado A
L1 = 1.00
t1 = 0.46
g_a = 2*L1 / (t1**2)
print("La aceleración de la gravedad es: ", g_a)

La aceleración de la gravedad es:  9.45179584120983


In [201]:
# Para el apartado B
L2 = 2.00
t2 = 0.66
g_b = 2*L2 / (t2**2)
print("La aceleración de la gravedad es: ", g_b)

La aceleración de la gravedad es:  9.182736455463727


In [202]:
# creamos un dataframe con los datos
df = pd.DataFrame({
    'Longitud': [L1, L2],
    'Tiempo': [t1, t2],
    'Gravedad': [g_a, g_b],
    'Planeta más cercano': ['Tierra', 'Saturno'],
    '2ndo más cercano': ['Saturno', 'Tierra']},index= ['Escenario A', 'Escenario B'])
df

,Longitud,Tiempo,Gravedad,Planeta más cercano,2ndo más cercano
Escenario A,1.0,0.46,9.451796,Tierra,Saturno
Escenario B,2.0,0.66,9.182736,Saturno,Tierra


In [203]:
def calcular_error(L, t):
    L_simbolo, t_simbolo = sp.symbols('L t')
    g_calculada_simbolo = (2*L_simbolo) / (t_simbolo**2)
    
    derivada_parcial_L = sp.diff(g_calculada_simbolo, L_simbolo)
    derivada_parcial_t = sp.diff(g_calculada_simbolo, t_simbolo)
    
    error_L = 0.01
    error_t = 0.01
    
    error_g_calculada = sp.sqrt((derivada_parcial_L * error_L)**2 + (derivada_parcial_t * error_t)**2)
    
    return float(error_g_calculada.subs({L_simbolo: L, t_simbolo: t}))

    

In [204]:
fake = fk.Faker()

def generar_datos(planetas, L_objetivo, t_objetivo):
    L = round(rd.uniform(L_objetivo - 0.1, L_objetivo + 0.1), 2)
    t = round(rd.uniform(t_objetivo - 0.1, t_objetivo + 0.1), 2)
    g_calculada = round((2*L) / (t**2), 2)

    diferencia_minima = float('inf')
    planeta_mas_cercano = ''
    
    diferencia_minima_2 = float('inf')
    segundo_planeta_mas_cercano = ''

    for planeta, gravedad in planetas.items():
        diferencia = abs(gravedad - g_calculada)
        if diferencia < diferencia_minima:
            diferencia_minima = diferencia
            planeta_mas_cercano = planeta
    
    for planeta, gravedad in planetas.items():
        if planeta != planeta_mas_cercano:
            diferencia = abs(gravedad - g_calculada)
            if diferencia < diferencia_minima_2:
                diferencia_minima_2 = diferencia
                segundo_planeta_mas_cercano = planeta
    
    error_g_calculada = calcular_error(L, t)
    error_porcentaje = (error_g_calculada / g_calculada) * 100
    
    return {
        'Id': fake.uuid4(),
        'Longitud (m)': L,
        'Tiempo (s)': t,
        'Gravedad (m/s^2)': g_calculada,
        'Planeta más cercano': planeta_mas_cercano,
        'Segundo Planeta más cercano': segundo_planeta_mas_cercano,
        'Error Gravedad (m/s^2)': error_g_calculada,
        'Error Porcentaje (%)': error_porcentaje
    }

dataframe_a = pd.DataFrame([generar_datos(planetas, L1, t1) for _ in range(1000)])
dataframe_a



,Id,Longitud (m),Tiempo (s),Gravedad (m/s^2),Planeta más cercano,Segundo Planeta más cercano,Error Gravedad (m/s^2),Error Porcentaje (%)
0,8f12682d-f1e6-4482-9443-2b980ea3a902,0.91,0.53,6.48,Urano,Saturno,0.254653,3.929833
1,6abb3d69-0bdd-4775-8d24-524bb157d870,1.08,0.41,12.85,Neptuno,Tierra,0.637996,4.964953
2,0cddd4a9-6ced-43e3-b704-2babc4d11b0e,0.98,0.50,7.84,Urano,Saturno,0.323643,4.128103
3,79d4f9df-a62e-4ceb-ade9-233f8b0c7656,1.01,0.40,12.62,Neptuno,Tierra,0.643507,5.099106
4,28c4d2ea-413c-4787-ba04-c819cf019367,0.90,0.49,7.50,Urano,Saturno,0.317130,4.228403
...,...,...,...,...,...,...,...,...
995,ee316fd1-92a7-4ec4-83dd-79e2627a82fd,0.99,0.49,8.25,Urano,Saturno,0.346748,4.203011
996,883b814d-3f07-45d8-ae8e-908d28c65483,0.97,0.45,9.58,Tierra,Saturno,0.437093,4.562562
997,0dd6ce70-b1c6-46fe-9819-da1debbff5d3,0.94,0.55,6.21,Urano,Marte,0.235468,3.791758
998,bbf1dbd9-0848-4a8c-a767-9a3184523640,1.07,0.55,7.07,Urano,Saturno,0.265611,3.756867


In [205]:
dataframe_b = pd.DataFrame([generar_datos(planetas, L2, t2) for _ in range(1000)])
dataframe_b

,Id,Longitud (m),Tiempo (s),Gravedad (m/s^2),Planeta más cercano,Segundo Planeta más cercano,Error Gravedad (m/s^2),Error Porcentaje (%)
0,65da9aa6-0647-41df-a486-5697b0bea286,1.93,0.63,9.73,Tierra,Saturno,0.312827,3.215078
1,b920e98e-e1c1-44eb-905f-3367218241a0,2.07,0.75,7.36,Urano,Saturno,0.199461,2.710072
2,4b889cfe-daa2-495d-b4a6-68c8665a8bc5,2.01,0.73,7.54,Urano,Saturno,0.210055,2.785872
3,648d7890-fe93-4d52-811d-d6e28b7e42dc,1.98,0.74,7.23,Urano,Saturno,0.198831,2.750079
4,24c1376e-3521-4e51-ae25-0772224b9bf4,1.98,0.60,11.00,Neptuno,Tierra,0.370852,3.371378
...,...,...,...,...,...,...,...,...
995,e2c7fa6c-5b1c-4a32-a62f-27f5e18a9a75,2.08,0.69,8.74,Urano,Saturno,0.256726,2.937365
996,28aad933-6760-42cb-ac7b-fad93a616f0e,2.08,0.63,10.48,Neptuno,Tierra,0.336531,3.211178
997,71533d29-3c79-4f18-ac23-536d6a244451,1.95,0.63,9.83,Tierra,Saturno,0.315985,3.214498
998,5ca2c7fb-aa4a-45ed-a5f9-7267fa5d1286,1.94,0.73,7.28,Urano,Saturno,0.202977,2.788146


In [206]:
def contar_planetas(dataframe):
    conteo_planetas = dataframe['Planeta más cercano'].value_counts()
    
    planeta_mas_cercana_nave = conteo_planetas.idxmax()
    
    conteo_planetas_2 = dataframe['2ndo más cercano'].value_counts()
    
    segundo_planeta_mas_cercana_nave = conteo_planetas_2.idxmax()
    
    return conteo_planetas, planeta_mas_cercana_nave, conteo_planetas_2, segundo_planeta_mas_cercana_nave


conteo_planetas, planeta_mas_cercana_nave, conteo_planetas_2, segundo_planeta_mas_cercana_nave = contar_planetas(dataframe)

print(conteo_planetas)
print(f'El planeta más cercano a la nave es: {planeta_mas_cercana_nave}')
print(f'El segundo planeta más cercano a la nave es: {segundo_planeta_mas_cercana_nave}')

Planeta más cercano
Júpiter    259
Neptuno    236
Marte      214
Urano      199
Tierra      62
Saturno     30
Name: count, dtype: int64
El planeta más cercano a la nave es: Júpiter
El segundo planeta más cercano a la nave es: Neptuno


In [207]:
# creamos un archivo excel con los datos
def crear_excel_exp_A():
    df = pd.DataFrame(dataframe_a)
    df.to_excel('Datos_experimentos_A.xlsx', index=False)
    print('Archivo A creado con éxito')


def crear_excel_exp_B():
    df = pd.DataFrame(dataframe_b)
    df.to_excel('Datos_experimentos_B.xlsx', index=False)
    print('Archivo  B creado con éxito')

crear_excel_exp_A()
crear_excel_exp_B()


Archivo A creado con éxito
Archivo  B creado con éxito


In [218]:
print(dataframe_a.describe())


       Longitud (m)   Tiempo (s)  Gravedad (m/s^2)  Error Gravedad (m/s^2)   
count   1000.000000  1000.000000       1000.000000             1000.000000  \
mean       1.001370     0.459670          9.964740                0.466740   
std        0.058124     0.058732          2.669692                0.182231   
min        0.900000     0.360000          5.800000                0.216860   
25%        0.950000     0.410000          7.735000                0.313899   
50%        1.000000     0.460000          9.490000                0.422065   
75%        1.050000     0.510000         12.000000                0.599312   
max        1.100000     0.560000         16.670000                0.938698   

       Error Porcentaje (%)  
count           1000.000000  
mean               4.538291  
std                0.561865  
min                3.682851  
25%                4.043576  
50%                4.462321  
75%                4.983516  
max                5.665124  


In [219]:
print(dataframe_b.describe())

       Longitud (m)   Tiempo (s)  Gravedad (m/s^2)  Error Gravedad (m/s^2)   
count   1000.000000  1000.000000       1000.000000             1000.000000  \
mean       1.998660     0.659090          9.420090                0.296405   
std        0.058513     0.057924          1.698345                0.078831   
min        1.900000     0.560000          6.610000                0.177452   
25%        1.950000     0.610000          7.957500                0.227597   
50%        2.000000     0.660000          9.180000                0.282196   
75%        2.050000     0.710000         10.800000                0.358269   
max        2.100000     0.760000         13.390000                0.482549   

       Error Porcentaje (%)  
count           1000.000000  
mean               3.099180  
std                0.267213  
min                2.673556  
25%                2.861846  
50%                3.072884  
75%                3.316544  
max                3.610650  


In [220]:
# histogramas
df.hist(column=['Longitud (m)', 'Tiempo (s)', 'Gravedad (m/s^2)'])
plt.show()

KeyError: "None of [Index(['Longitud (m)', 'Tiempo (s)', 'Gravedad (m/s^2)'], dtype='object')] are in the [columns]"

In [ ]:
# grafico de dispersion
plt.scatter(df['Longitud (m)'], df['Tiempo (s)'])
plt.xlabel('Longitud (m)')
plt.ylabel('Tiempo (s)')
plt.show()

In [ ]:
# codificar etiquetas de planetas 
codificador_etiquetas = LabelEncoder()
df['Planeta más cercano (codigo)'] = codificador_etiquetas.fit_transform(df['Planeta más cercano'])
df['2ndo más cercano (codigo)'] = codificador_etiquetas.fit_transform(df['2ndo más cercano'])

In [ ]:
# correlación entre variables 
df_numeric = df.select_dtypes(include=[np.number])
print(df_numeric.corr())


In [ ]:
# regresion lineal 
X = df['Longitud (m)'].values.reshape(-1, 1)
y = df['Tiempo (s)'].values.reshape(-1, 1)
regresion_lineal = LinearRegression()
regresion_lineal.fit(X, y)

y_pred = regresion_lineal.predict(X)
plt.scatter(X, y, color='blue') 
plt.plot(X, y_pred, color='red')
plt.xlabel('Longitud (m)')
plt.ylabel('Tiempo (s)')
plt.show()

In [ ]:

def analizar_dataframe(df):
    print(df.describe())
    
    df.hist(column=['Longitud (m)', 'Tiempo (s)', 'Gravedad (m/s^2)'])
    plt.show()
    
    plt.scatter(df['Longitud (m)'], df['Tiempo (s)'])
    plt.xlabel('Longitud (m)')
    plt.ylabel('Tiempo (s)')
    plt.show()
    
    codificador_etiquetas = LabelEncoder()
    df['Planeta más cercano (codigo)'] = codificador_etiquetas.fit_transform(df['Planeta más cercano'])
    df['Segundo Planeta más cercano (codigo)'] = codificador_etiquetas.fit_transform(df['Segundo Planeta más cercano'])
    
    df_numeric = df.select_dtypes(include=[np.number])
    print(df_numeric.corr())
    
    X = df['Longitud (m)'].values.reshape(-1, 1)
    y = df['Tiempo (s)'].values.reshape(-1, 1)

    regresion_lineal = LinearRegression()
    regresion_lineal.fit(X, y)

    y_pred = regresion_lineal.predict(X)
    plt.scatter(X, y, color='blue') 
    plt.plot(X, y_pred, color='red')
    plt.xlabel('Longitud (m)')
    plt.ylabel('Tiempo (s)')
    plt.show()

def contar_planetas(dataframe):
    conteo_planetas = dataframe['Planeta más cercano'].value_counts()
    
    planeta_mas_cercana_nave = conteo_planetas.idxmax()
    
    conteo_planetas_2 = dataframe['Segundo Planeta más cercano'].value_counts()
    
    segundo_planeta_mas_cercana_nave = conteo_planetas_2.idxmax()
    
    return conteo_planetas, planeta_mas_cercana_nave, conteo_planetas_2, segundo_planeta_mas_cercana_nave

analizar_dataframe(dataframe_a)
conteo_planetas_a, planeta_mas_cercana_nave_a, conteo_planetas_2_a, segundo_planeta_mas_cercana_nave_a = contar_planetas(dataframe_a)

print(conteo_planetas_a)
print(f'El planeta más cercano a la nave es: {planeta_mas_cercana_nave_a}')
print(f'El segundo planeta más cercano a la nave es: {segundo_planeta_mas_cercana_nave_a}')

analizar_dataframe(dataframe_b)
conteo_planetas_b, planeta_mas_cercana_nave_b, conteo_planetas_2_b, segundo_planeta_mas_cercana_nave_b = contar_planetas(dataframe_b)

print(conteo_planetas_b)
print(f'El planeta más cercano a la nave es: {planeta_mas_cercana_nave_b}')
print(f'El segundo planeta más cercano a la nave es: {segundo_planeta_mas_cercana_nave_b}')

dataframe_combinado = pd.concat([dataframe_a, dataframe_b], ignore_index=True)

analizar_dataframe(dataframe_combinado)
conteo_planetas_combinado, planeta_mas_cercana_nave_combinado, conteo_planetas_2_combinado, segundo_planeta_mas_cercana_nave_combinado = contar_planetas(dataframe_combinado)

print(conteo_planetas_combinado)
print(f'El planeta más cercano a la nave es: {planeta_mas_cercana_nave_combinado}')
print(f'El segundo planeta más cercano a la nave es: {segundo_planeta_mas_cercana_nave_combinado}')